**Hyper-heuristiques de Sélection  : Approche perturbatrice**

  **Premier Cas :**
  
  l'idée est **d'utiliser Hill Climbing, Tabu Search, et Simulated Annealing** comme heuristiques de bas niveau, combinées avec **Random Gradient** comme  stratégie de selection et une strategie **move acceptance** qui n'accepte que les solutions qui améliorent la solution actuelle.




# **Low-level heuristics**
1.  Hill Climbing
2. Tabu Search, and
3. Simulated Annealing


## **Comparaison**

| Aspect                  | Hill Climbing                                                                 | Recherche Tabou | Simulated Annealing|
|-------------------------|---------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|---------------------------------------|
| **Plus rapide**         | Oui                                                                                       | Non                                                                                        | Non                                   |
| **Meilleure solution**  | Non                                                                                      | Oui                                                                                     | Oui                                  |
| **Évite les optima locaux** | Non                                                                                     | Oui                                                                                     | Oui                                  |



## **Implementation**

In [ ]:
import random
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [ ]:
# Définition de la fonction pour emballer les articles de manière aléatoire dans des bins
def random_packing(items, bin_capacity):
    # Liste pour stocker les bins
    bins = []
    # Mélangez les articles au hasard
    random.shuffle(items)

    # Pour chaque article, créez une nouvelle bin contenant cet article
    for item in items:
        bins.append([item])

    # Retourne la liste des bins
    return bins

# Fonction pour calculer la fitness d'une solution
def fitness(bins):
    # La fitness est simplement le nombre de bins utilisés
    return len(bins)

# Fonction pour générer un voisin d'une solution donnée
def generate_neighbor(solution, bin_capacity):
    # Créez une copie de la solution actuelle
    neighbor = [bin[:] for bin in solution]
    neighborT = [bin[:] for bin in solution]

    # Sélectionnez deux bins au hasard
    bin1_index, bin2_index = random.sample(range(len(neighbor)), 2)

    # Choisissez un article au hasard dans la première bin
    item1_index = random.randint(0, len(neighborT[bin1_index]) - 1)
    item1 = neighborT[bin1_index].pop(item1_index)

    # Choisissez un autre article au hasard dans la seconde bin
    item2_index = random.randint(0, len(neighborT[bin2_index]) - 1)
    item2 = neighborT[bin2_index].pop(item2_index)

    # Si le transfert de l'article 1 à la seconde bin ne dépasse pas la capacité de la bin,
    # effectuez le transfert et vérifiez si la première bin est maintenant vide
    if (sum(neighbor[bin2_index]) + item1 < bin_capacity):
        neighbor[bin1_index] = neighborT[bin1_index]
        neighbor[bin2_index].append(item1)
        if len(neighbor[bin1_index]) == 0:
            neighbor.remove(neighbor[bin1_index])

    # Sinon, si le transfert de l'article 2 à la première bin ne dépasse pas la capacité de cette dernière,
    # effectuez le transfert et vérifiez si la seconde bin est maintenant vide
    elif (sum(neighbor[bin1_index]) + item2 < bin_capacity):
        neighbor[bin2_index] = neighborT[bin2_index]
        neighbor[bin1_index].append(item2)
        if len(neighbor[bin1_index]) == 0:
            neighbor.remove(neighbor[bin2_index])

    # Retourne la solution voisine
    return neighbor


In [ ]:
def hill_climbing(initial_solution, bin_capacity, max_iterations):
    current_solution = initial_solution
    current_fitness = fitness(current_solution)

    # Itère jusqu'à ce qu'un nombre maximal d'itérations soit atteint
    for iteration in range(max_iterations):
        # Trouve une solution voisine
        neighbor_solution = generate_neighbor(current_solution, bin_capacity)

        # Calcule la fitness de la solution voisine
        neighbor_fitness = fitness(neighbor_solution)

        # Si la solution voisine est meilleure, mettez à jour la solution actuelle
        if neighbor_fitness <= current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness

    return current_solution, current_fitness


In [ ]:
def tabu_search(initial_solution, bin_capacity, max_iterations, tabu_size):
    current_solution = initial_solution
    current_fitness = fitness(current_solution)
    tabu_list = []  # Initialise la liste tabou
    best_neighbor_solution = current_solution
    best_neighbor_fitness = current_fitness

    # Itère jusqu'à ce qu'un nombre maximal d'itérations soit atteint
    for iteration in range(max_iterations):
        # Trouve la meilleure solution voisine

        for _ in range(10):  # Essayez 10 voisins au hasard
            neighbor_solution = generate_neighbor(current_solution, bin_capacity)
            neighbor_fitness = fitness(neighbor_solution)

            if neighbor_fitness <= best_neighbor_fitness and neighbor_solution not in tabu_list:
                best_neighbor_solution = neighbor_solution
                best_neighbor_fitness = neighbor_fitness

        # Mettez à jour la solution actuelle et la fitness
        current_solution = best_neighbor_solution
        current_fitness = best_neighbor_fitness

        # Mettez à jour la liste tabou
        tabu_list.append(best_neighbor_solution)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)  # Supprime la solution la plus ancienne de la liste tabou

    # Affiche l'itération où il s'est arrêté
    # print(f"Arrêté à l'itération: {iteration + 1}")

    return current_solution, current_fitness


In [ ]:
def simulated_annealing(initial_solution, bin_capacity, max_iterations, initial_temperature, cooling_rate=0.99):
    current_solution = initial_solution
    current_fitness = fitness(current_solution)
    best_solution = current_solution
    best_fitness = current_fitness
    temperature = initial_temperature

    # Itère jusqu'à ce qu'un nombre maximal d'itérations soit atteint
    for iteration in range(max_iterations):
        # Trouve une solution voisine
        neighbor_solution = generate_neighbor(current_solution, bin_capacity)

        # Calcule la fitness de la solution voisine
        neighbor_fitness = fitness(neighbor_solution)

        # Si la solution voisine est meilleure, mettez à jour la solution actuelle
        if neighbor_fitness <= current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
            # Vérifie si la voisine est la meilleure solution trouvée jusqu'à présent
            if current_fitness <= best_fitness:
                best_solution = current_solution
                best_fitness = current_fitness
        else:
            # Si la solution voisine est pire, l'accepter avec une probabilité basée sur la température
            delta_fitness = neighbor_fitness - current_fitness
            acceptance_probability = math.exp(-delta_fitness / temperature)
            if random.random() < acceptance_probability:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness

        # Refroidit la température
        temperature *= cooling_rate

    # Affiche l'itération où il s'est arrêté
    # print(f"Arrêté à l'itération: {iteration + 1}")

    return best_solution, best_fitness


# **High Level**


## **Random Gradient strategy for selection**
une LLH est choisi aléatoirement et
est appliquée répétitivement tant qu’elle améliore la
solution courante.

In [ ]:
def evaluate(bins):
    return len(bins)

In [ ]:
def move_acceptance( candidate_fitness,  best_fitness):
    # Accept the candidate solution if it improves upon the current best solution
    if candidate_fitness < best_fitness:
        return True
    else:
        return False


In [ ]:
def random_descent_strategy(problem_state, bin_capacity, heuristics, max_iterations):
    # Initialise la meilleure solution et sa fitness
    best_solution = problem_state
    best_fitness = evaluate(problem_state)
    heuristic_choices = []

    for iteration in range(max_iterations):

        # Sélectionne au hasard une heuristique parmi celles disponibles
        heuristic, params = random.choice(list(heuristics.items()))
        heuristic_choices.append(heuristic.__name__)

        # Applique la heuristique sélectionnée répétitivement tant qu'elle améliore la solution
        improvement = True
        while improvement:
            candidate_solution, candidate_fitness = heuristic(best_solution, bin_capacity, 300, **params)

            print("-------------------------------")
            print("----- Heuristique utilisée :", heuristic.__name__)
            print("Nombre de bins actuellement : ", candidate_fitness)
            print(candidate_solution)

            if move_acceptance(candidate_fitness, best_fitness):
                best_solution = candidate_solution
                best_fitness = candidate_fitness
            else:
                improvement = False

    return best_solution , heuristic_choices


# **Principe de fonctionnement** :
Une solution initiale est générée. A chaque itération, on sélectionne
une Recherche Locale à appliquer sur la solution en utilisant la methode de selection "Random Gradient". La
solution candidate est évaluée et envoyée en paramètres à la
fonction d’acceptation du pas. Cette fonction va comparer la qualité
de la solution actuelle avec la solution candidate. Si elle est
supérieure, la solution candidate est acceptée et on passe à
l’itération suivante, sinon, la solution actuelle est gardée.

In [29]:
if __name__ == "__main__":
    # Ouvre le fichier contenant les données du problème
    with open("/content/BPP_1000_1000_0.1_0.7_0.txt", "r") as file:
        # Lit le nombre d'objets
        num_objects = int(file.readline().strip())
        # Lit la capacité de la bin
        bin_capacity = int(file.readline().strip())
        # Lit les objets, ligne par ligne
        items = [int(line.strip()) for line in file]

    # Dictionnaire contenant les heuristiques disponibles
    heuristics = {
        hill_climbing: {},
        simulated_annealing: {'initial_temperature': 100, 'cooling_rate': 0.99},
        tabu_search: {'tabu_size': 5}
    }

    # Génère une solution initiale en utilisant la stratégie d'emballage aléatoire
    initial_solution = random_packing(items, bin_capacity)

    print("solution initiale")
    print(initial_solution)
    # Utilise la solution initiale comme état du problème
    problem_state = initial_solution
    # Applique la stratégie de descente aléatoire pour trouver la meilleure solution
    start_time = time.time()
    best_problem_state, heuristic_choices = random_descent_strategy(problem_state, bin_capacity, heuristics, 20)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("---------------------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------------------")
    # Affiche la meilleure solution trouvée
    print("-------------meilleure solution--------------------------------------------------------------")
    print("---------------------------------------------------------------------------------------------")
     # Affiche le nombre de bins utilisés pour la meilleure solution
    print("nombre de bins:")
    print(evaluate(best_problem_state))
    print("---------------------------------------------------------------------------------------------")
    print(best_problem_state)
    print("Time taken for execution (in seconds):", elapsed_time)
    print("Heuristics choises")
    print(heuristic_choices)







solution initiale
[[205], [490], [319], [159], [347], [375], [653], [505], [455], [267], [395], [603], [166], [215], [407], [627], [627], [600], [432], [462], [448], [624], [427], [487], [673], [170], [426], [700], [677], [212], [685], [647], [128], [156], [424], [171], [601], [364], [689], [124], [116], [172], [447], [385], [340], [535], [249], [253], [378], [190], [620], [333], [277], [565], [249], [248], [436], [667], [356], [526], [106], [247], [211], [486], [427], [672], [115], [543], [389], [327], [635], [323], [599], [113], [105], [457], [594], [595], [700], [149], [364], [667], [134], [155], [182], [541], [116], [220], [658], [432], [531], [523], [232], [342], [416], [393], [232], [615], [462], [186], [346], [586], [275], [416], [596], [581], [612], [633], [152], [242], [282], [687], [581], [275], [118], [634], [680], [111], [589], [183], [550], [261], [212], [697], [468], [225], [699], [593], [289], [270], [486], [258], [683], [394], [522], [574], [188], [401], [226], [454], [

 **Deuxiéme Cas  :**

 L'idée est de combiner le hill climbing avec le recuit simulé. Le hill climbing est rapide mais tombe souvent dans le premier optimum local du voisinage. En revanche, le recuit simulé est moins rapide mais peut échapper aux optima locaux en acceptant, avec une certaine probabilité, des solutions moins bonnes que l'actuelle. En combinant ces deux heuristiques, on bénéficie des avantages de chacune. Cette combinaison est guidée par une stratégie de haut niveau utilisant l'apprentissage par renforcement.

**Low-level heuristics:**


1.   hill climbing
2.   recuit simulé



**High-level strategic:**

l'approche adoptée implique l'utilisation d'un apprentissage par renforcement. L'idée centrale consiste à attribuer un score à chaque heuristique. À chaque itération, le choix de l'heuristique se base sur ce score, avec une certaine probabilité. Initialement, toutes les heuristiques ont un score nul. À chaque itération, ces scores sont mis à jour en fonction de l'amélioration par rapport à la solution précédente. Les heuristiques qui améliorent la solution sont récompensées, tandis que celles qui ne le font pas sont pénalisées

In [ ]:
import random
import math
import time

class Bin:
    def __init__(self, capacity):
        # Initialisation d'une nouvelle "bin" avec une capacité donnée et une liste vide d'éléments.
        self.capacity = capacity
        self.items = []

    def add_item(self, item):
        # Ajout d'un élément à la "bin".
        self.items.append(item)

    def remaining_capacity(self):
        # Calcul de la capacité restante dans la "bin" en soustrayant la somme des tailles des éléments actuels de sa capacité totale.
        return self.capacity - sum(self.items)

class Item:
    def __init__(self, size):
        # Initialisation d'un nouvel objet "Item" avec une taille donnée.
        self.size = size

def initial_solution(items, bin_capacity):
    # Création d'une solution initiale en plaçant chaque élément dans la première "bin" disponible pouvant le contenir, sinon en créant une nouvelle "bin".
    bins = [Bin(bin_capacity)]
    for item_size in items:
        for bin in bins:
            if bin.remaining_capacity() >= item_size:
                bin.add_item(item_size)
                break
        else:
            new_bin = Bin(bin_capacity)
            new_bin.add_item(item_size)
            bins.append(new_bin)
    return bins

def evaluate_solution(bins):
    # Évaluation de la solution en comptant le nombre total de "bins" utilisées.
    return len(bins)

def hill_climbing(bins):
    # Algorithme de recherche locale "hill climbing" pour améliorer la solution en déplaçant les éléments entre les "bins" si cela permet de réduire le nombre total de "bins" utilisées.
    for i in range(len(bins)):
        for j in range(len(bins)):
            if i != j:
                for item in bins[i].items:
                    if bins[j].remaining_capacity() >= item:
                        bins[i].items.remove(item)
                        bins[j].add_item(item)
                        return bins
    return bins

def simulated_annealing(bins, temperature):
    # Algorithme de recuit simulé pour permettre des mouvements non-optimaux dans l'espoir d'éviter les minimas locaux.
    new_bins = bins[:]
    for i in range(len(new_bins)):
        for j in range(len(new_bins)):
            if i != j:
                for item in new_bins[i].items:
                    if new_bins[j].remaining_capacity() >= item:
                        new_bins[i].items.remove(item)
                        new_bins[j].add_item(item)
                        if evaluate_solution(new_bins) < evaluate_solution(bins):
                            return new_bins
                        elif random.uniform(0, 1) < math.exp((evaluate_solution(bins) - evaluate_solution(new_bins)) / temperature):
                            return new_bins
                        else:
                            new_bins[j].items.remove(item)
                            new_bins[i].add_item(item)
    return bins

class ReinforcementLearningAgent:
    def __init__(self, temperature_decay=0.99):
        # Initialisation de l'agent d'apprentissage par renforcement avec les paramètres par défaut.
        self.epsilon = 0.1
        self.learning_rate = 0.1
        self.temperature_decay = temperature_decay
        self.q_values = {'hill_climbing': 0, 'simulated_annealing': 0}

    def choose_action(self):
        # Choix d'une action par l'agent selon une politique d'exploration/exploitation.
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['hill_climbing', 'simulated_annealing'])
        else:
            return max(self.q_values, key=self.q_values.get)

    def update_q_values(self, action, reward):
        # Mise à jour des valeurs Q en utilisant la règle de mise à jour de Q-learning.
        self.q_values[action] += self.learning_rate * (reward - self.q_values[action])

def main():
    # Définition des paramètres du problème.
    bin_capacity = 50
    items = [35, 35, 34, 34, 34, 33, 33, 33, 32, 31, 31, 30, 30, 29, 28, 28, 28, 27, 26, 26, 26, 26, 25, 25, 22, 22, 21, 20, 18, 18, 16, 16, 16, 16, 16, 13, 13, 13, 12, 11, 11, 10, 10, 9, 9, 7, 7, 7, 7, 5]

    # Initialisation de l'agent et de l'environnement.
    agent = ReinforcementLearningAgent()
    bins = initial_solution(items, bin_capacity)
    temperature = 100.0

    # Boucle principale
    start_time = time.time()
    for episode in range(1000):
        action = agent.choose_action()
        if action == 'hill_climbing':
            prev_bins = bins[:]
            bins = hill_climbing(bins)
            reward = evaluate_solution(bins)
            if reward < evaluate_solution(prev_bins):
                agent.update_q_values(action, 1)  # Récompense positive pour l'amélioration.
            else:
                agent.update_q_values(action, -1)  # Récompense négative pour l'absence d'amélioration ou la détérioration.
        else:
            prev_bins = bins[:]
            bins = simulated_annealing(bins, temperature)
            reward = evaluate_solution(bins)
            if reward < evaluate_solution(prev_bins):
                agent.update_q_values(action, 1)  # Récompense positive pour l'amélioration.
            else:
                agent.update_q_values(action, -1)  # Récompense négative pour l'absence d'amélioration ou la détérioration.

        temperature *= agent.temperature_decay

    end_time = time.time()
    execution_time = end_time - start_time
    print("Number of bins used:", evaluate_solution(bins))
    print("Execution time:", execution_time)

if __name__ == "__main__":
    main()


Number of bins used: 23
Execution time: 0.008333206176757812


**Conclusion:**
Nous avons exécuté le même benchmark sur les deux solutions proposées, et elles ont produit une résolution proche de la solution optimale **(23 bins avec un benchmark de 50 items )** obtenue par la méthode de branch and bound, mais avec un temps d'exécution beaucoup plus court.